In [ ]:
# RISEN, like RTF, provides a framework for you to articulate your request to the LLM.

# R - Role - the role the LLM should embody
# I - Input - the request, or root of the prompt intention
# S - Steps - steps for the LLM to follow when addressing the input
# E - Expected Output - a description of what you are expecting the model to return
# N - Nuance - nuance and other considerations that should be factored into the response

In [ ]:
from langchain_aws import ChatBedrock
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import boto3

In [ ]:
# looks up the id for the guardrail named 'sanitize'
bedrock_client = boto3.client('bedrock')
guardrails = bedrock_client.list_guardrails()
this_guardrail_id = None
for guardrail in guardrails['guardrails']:
    if guardrail['name'] == 'sanitizer':
        this_guardrail_id = guardrail['id']
        break
print("guardrail ID: " + this_guardrail_id)

# reference to the foundation model via amazon bedrock.
# https://docs.aws.amazon.com/bedrock/latest/userguide/inference-parameters.html
model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # claude is awesome!    
    model_kwargs={
        "temperature": 0.95,
        "top_p": 0.95,
        "top_k": 250
    },
    guardrails={                                         # guardrails filter prompts and responses
        "guardrailIdentifier": this_guardrail_id, 
        "guardrailVersion": "1"
    }
)

In [ ]:
role = PromptTemplate(
    template="""
Act as an experienced travel consultant who creates customized travel plans
suited specifically to the needs and requests of the client. You apply extra
effort to finding unique experiences for the client.
"""
)

In [ ]:
input_ = PromptTemplate(
    input_variables = ['text'],
    template="""
{text}

A client wants a unique 5-day itinerary for a family vacation in Tucson, Arizona, 
including a mix of popular attractions and off-the-beaten-path experiences.
"""
)

In [ ]:
# remember.. the LLM does not actually *follow* the steps
steps = PromptTemplate(
    input_variables = ['text'],
    template="""
{text}

1. Identify popular attractions that are family-friendly.
2. Research lesser-known spots that offer unique experiences.
3. Combine these into a day-by-day itinerary, balancing both types of activities.
4. Include brief descriptions and reasons for each recommendation.
"""
)

In [ ]:
expected_output = PromptTemplate(
    input_variables = ['text'],
    template="""
{text}

A detailed 5-day itinerary for a family vacation in Newark, New Jersey, 
presented in a structured format with headings for each day and bullet points for activities
"""
)

In [ ]:
# it's best to affirm your requirements, as opposed to telling the LLM what you *don't* want.
nuance = PromptTemplate(
    input_variables = ['text'],
    template="""
{text}

Ensure the daytime activities are suitable for all family members, 
including children and older adults, and consider accessibility and convenience.

Nighttime activities should be tailored to adult interests.
"""
)

In [ ]:
chain = role | input_ | steps | expected_output | nuance | model | StrOutputParser()

In [ ]:
print(chain.invoke({}))